#### Importando as bibliotecas

In [1]:
import pandas as pd

#### Importando os dados

In [2]:
df = pd.read_csv('./data/TA_PRECO_MEDICAMENTO.CSV', sep = ';', encoding='latin-1', on_bad_lines='skip')

C:\Users\jose-\AppData\Local\Temp\ipykernel_12564\1792964528.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/TA_PRECO_MEDICAMENTO.CSV', sep = ';', encoding='latin-1', on_bad_lines='skip')


### Capturando as colunas importantes
<b>Tipo de produto (status do produto)</b>: Indica qual se a venda do medicamento necessita de recita.<br>
<b>Tarja</b>: indica qual se a venda do medicamento necessita de recita, sendo classificado em Tarja preta, Tarja vermelha ou Venda livre.<br>
<b>PF 18% alc</b>: preço de fábrica.<br>
<b>PMC 18% alc</b>: preço máximo ao consumidor (variável alvo).<br>
<br>
<b>Obs:</b> Foram escolhidos os PF e PMC com alíquota de 18% ao invés dos outros, pois este é o percentual aplicado na maioria dos estados brasileiros.

In [3]:
# Deixa os nomes das colunas em letras minúsculas
df.rename(columns = {column:column.lower() for column in df.columns}, inplace = True)

# Simplifica nomes das colunas
df.rename(columns = {'tipo de produto (status do produto)': 'tipo_produto',
                    'pf 18% alc': 'pf_18',
                    'pmc 18% alc': 'pmc_18'}, inplace = True)

df = df[['tipo_produto', 'tarja', 'pf_18','pmc_18']]

In [4]:
# Retira linhas com dados nulos

df.dropna(inplace = True)

In [5]:
df.head()

,tipo_produto,tarja,pf_18,pmc_18
0,Similar,Tarja Vermelha (**),44,"60,83"
1,Similar,Tarja Preta (**),"6,15","8,5"
2,-,Tarja Vermelha,"130,95","181,03"
3,Novo,Tarja Vermelha (**),"60,4","83,5"
4,Novo,Tarja Vermelha (**),"60,4","83,5"


### Consertando os dados
Removeremos do dataset as linhas que não contém significado na coluna 'tipo_produto'.<br> 

Na coluna 'tarja' simplificaremos às tarjas vermelha, preta e livre.<br>

Na coluna 'pf_18' e 'pmc_18' faremos a conversão para o tipo float.<br>

In [6]:
# Corrige dataset a partir da coluna 'tipo_produto'

df = df[df['tipo_produto'].str.contains('-') == False]
df['tipo_produto'].value_counts()

Genérico        8069
Similar         7885
Novo            3835
Específico      1435
Biológico        822
Fitoterápico     389
Name: tipo_produto, dtype: int64

In [7]:
# Simplifica coluna 'tarja'

def simplifica_tarja(tarja):
    
    if 'erm' in tarja:
        return 'Tarja vermelha'
    elif 'ret' in tarja:
        return 'Tarja preta'
    else:
        return 'Sem tarja'
    
df['tarja'] = df['tarja'].apply(simplifica_tarja)
df['tarja'].value_counts()

Tarja vermelha    17765
Sem tarja          3292
Tarja preta        1378
Name: tarja, dtype: int64

In [8]:
# Converte PF e PMC para o tipo float

def converte_float(preco):
    
    if ',' in preco:
        preco = preco.replace(',', '.')
    
    preco = float(preco)
    return preco

df['pf_18'] = df['pf_18'].apply(converte_float)
df['pmc_18'] = df['pmc_18'].apply(converte_float)

In [9]:
# Visão do dataset consertado

df.sample(5)

,tipo_produto,tarja,pf_18,pmc_18
20892,Específico,Tarja preta,38.11,52.68
18658,Genérico,Tarja vermelha,49.48,68.40
11115,Similar,Tarja vermelha,85.05,117.58
3699,Específico,Tarja preta,60.56,83.72
3930,Similar,Tarja vermelha,220.28,304.52
